# Most Dangerous Streets

In [1]:
import pytz
import folium
import pandas as pd
from datetime import datetime, timedelta
from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
start_date = pytz.timezone('America/Denver').localize(datetime(2023,1,1))
end_date = pytz.timezone('America/Denver').localize(datetime(2023,2,1))
most_recent_crash = cda.most_recent_crash_timestamp()

In [4]:
query = f"""
select
sr.gid
, sr.lrsroute
, st_length(sr.geom_denver) / 5280 as length_miles
, count(*) as num_crashes

from street_routes sr
inner join crashes c on st_dwithin(sr.geom_denver, c.geom_denver, 25)

where reported_date at time zone 'America/Denver' > '{start_date.strftime('%Y-%m-%d')}'
and reported_date at time zone 'America/Denver' < '{end_date.strftime('%Y-%m-%d')}'

group by 1,2,3
"""

street_crashes = pd.read_sql(query, cda.conn)
street_crashes

,gid,lrsroute,length_miles,num_crashes
0,4138,GIRARD AV3,0.863561,1
1,4937,W4TH AV6,0.740691,1
2,4490,W27TH AV5,0.496099,1
3,520,49TH AV5,1.369385,5
4,7336,WMEXICO AV2,1.252253,1
...,...,...,...,...
709,3888,SANTAFE DR,1.732911,6
710,7126,WGILL PL2,0.529357,1
711,2944,TREMONT PL3,0.435570,2
712,4220,18TH ST4,0.843447,3


In [5]:
days_in_data = (end_date - start_date).days

In [6]:
street_crashes['days_between_crashes'] = days_in_data / street_crashes['num_crashes']

In [10]:
street_crashes['crashes_per_mile_per_week'] = (
    (street_crashes['num_crashes'] / street_crashes['length_miles']) / (days_in_data/7))

In [13]:
street_crashes[street_crashes.num_crashes >= 10].sort_values(
    by='crashes_per_mile_per_week', ascending=False).head(10)

,gid,lrsroute,length_miles,num_crashes,days_between_crashes,crashes_per_mile_per_week
556,5588,PARKWB AV,0.651755,11,2.818182,3.811048
690,4225,20TH ST,1.453330,19,1.631579,2.952064
354,3635,WASHINGTOST1,1.134570,11,2.818182,2.189261
585,6796,WALAMEDA AV,3.508448,29,1.068966,1.866463
161,2127,PEORIA ST1,2.089566,17,1.823529,1.837085
398,3900,SPEERN BD,2.634723,20,1.550000,1.714081
395,918,CENPASB BD2,1.688449,12,2.583333,1.604832
222,880,BROADWAY3,3.331934,21,1.476190,1.423178
219,5379,WCOLFAX AV,3.567320,22,1.409091,1.392570
545,1540,I25NB FW,15.089276,90,0.344444,1.346823
